# Compute and compare different radial potentials

In [1]:
using JAC


We shall here describe how different atomic (radial) potentials can be calculated and applied within the JAC toolbox.

Any (radial) potential is handled in JAC by the data struct `Radial.Potential` that (may) contain a name, the radial function $Z(r)$ as well as the radial grid on which the potential is defined. The potential function `Zr` typically contains the effective charge $Z(r)$ as seen by an electron, so that the full radial potential is given by $V = -\:Z(r) / r$. This convention has been frequently applied in atomic physics and has also been adopted in JAC.


In [2]:
? Radial.Potential

`struct  Radial.Potential`  ... defines a struct for a local radial potential.

```
+ name           ::String            ... A name for the potential.
+ Zr             ::Array{Float64,1}  ... radial potential function Z(r) = - r * V(r) as usual in atomic structure theory.
+ grid           ::RadialGrid        ... radial grid on which the potential is defined.
```

---

`Radial.Potential()`  ... constructor to define an 'empty' instance of the radial potential.



Perhaps, the simplest choice of a radial potential is the **nuclear** potential which, of course, depends on the nuclear model. Let us first simply consider a $\;^{12}$C $\;$ carbon atom with a Fermi-distributed nuclear charge and a standard radial grid.


In [3]:
grid     = Radial.Grid(true)
nucModel = Nuclear.Model(6., "Fermi")

Fermi nuclear model for Z = 6.0 with mass = 12.18, radius R = 2.4934845703596404 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0.


Then, the corresponding nuclear potential is generated by simply calling:


In [4]:
nucPotential = Nuclear.nuclearPotential(nucModel, grid)

nuclear-potential: Fermi-distributed (radial) potential ... defined on 392 grid points ...
Zr:    [0.0, 0.015619818207380716, 0.035905804451892086, 0.060430845202356614, 0.08495453726302814, 0.10523736995823374, 0.11777879390742439, 0.12582486891780031, 0.14610064533238984, 0.17888509371708336  …  0.31112937694872056, 0.32313641267043025, 0.35185818903226784, 0.39828287685804126, 0.4543674906380097, 0.5103989493317181, 0.5566993479831148, 0.5853078885893802, 0.6022668932973185, 0.6428176395098482]  ...  [6.0, 6.000000000000002, 6.000000000000001, 6.000000000000002, 6.000000000000001, 6.0, 6.000000000000001, 6.0, 6.000000000000002, 6.000000000000002  …  6.000000000000002, 6.000000000000001, 6.000000000000002, 6.000000000000002, 6.000000000000001, 6.000000000000001, 6.0, 6.000000000000001, 6.000000000000001, 6.000000000000001]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 392,  ntL = 69,  ntS = 71, orderL = 7,  orderS = 8,  nsL = 62,  nsS = 63,  mesh = JAC.Radial.MeshGL()


As seen from the output, the effective charge Z(r) vanishes near to the origin but quickly increases to the full charge $Z = 6.\;$ at larger $r$-values, i.e. near to the surface and outside of the nucleus

To calculate **atomic** potentials, we also need to know the electron density around the nucleus. Most often, this density refers to a particular atomic level. Therefore, we first need to compute some of the low-lying levels of, for instance, neutral carbon. We shall here just compute the levels of the $1s^2 2s^2 2p^2$ ground-state configuration. We use the *simplest* approximation for these computation but will explain in some later tutorials how we could control and refine such simple structure calculations, if needed.


In [5]:
basis     = Basics.performSCF([Configuration("1s^2 2s^2 2p^2")], nucModel, grid, AsfSettings())
multiplet = Basics.performCI(basis, nucModel, grid, AsfSettings());


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^0 2p_3/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^1 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 6.0 with mass = 12.18, radius R = 2.4934845703596404 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.80086332e+01    -1.80086350e+01    +1.02396837e-07    
      2      2s_1/2      -4.50269966e+00    -4.50269857e+00    -2.42835410e-07    
  

     21     22p_3/2      +1.24996175e-01    -3.71915928e-02    +1.29754185e+00    
     22     23p_3/2      +2.69273197e+00    -3.40277908e-02    +1.01263690e+00    
     23     24p_3/2      +9.55804164e+00    -3.12511703e-02    +1.00326962e+00    
     24     25p_3/2      +2.55081379e+01    -2.88010381e-02    +1.00112909e+00    
     25     26p_3/2      +6.04337789e+01    -2.66281441e-02    +1.00044062e+00    
     26     27p_3/2      +1.34480686e+02    -2.46921860e-02    +1.00018361e+00    
     27     28p_3/2      +2.88122942e+02    -2.29599276e-02    +1.00007969e+00    
     28     29p_3/2      +6.01200790e+02    -2.14037625e-02    +1.00003560e+00    
     29     30p_3/2      +1.22657485e+03    -2.00006072e-02    +1.00001631e+00    
     30     31p_3/2      +2.44237088e+03    -1.87310403e-02    +1.00000767e+00    
     31     32p_3/2      +4.71496718e+03    -1.75786269e-02    +1.00000373e+00    
     32     33p_3/2      +8.74391356e+03    -1.65293840e-02    +1.00000189e+00    
    

  1s_1/2::  en [a.u.] = -9.8875981e+00;   self-cons'cy = 6.0317e-07  [6.3336e-05 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -4.5750453e-01;   self-cons'cy = 4.7378e-06  [6.3336e-05 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.5780785e-01;   self-cons'cy = 2.5151e-07  [5.1385e-07 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.5749168e-01;   self-cons'cy = 3.4875e-06  [3.4875e-06 for sym-block kappa = -2]

Iteration 15 for symmetries ... 
  1s_1/2::  en [a.u.] = -9.8875999e+00;   self-cons'cy = 9.4910e-08  [9.7914e-06 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -4.5750501e-01;   self-cons'cy = 5.2256e-07  [9.7914e-06 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.5780707e-01;   self-cons'cy = 2.4696e-06  [2.4696e-06 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.5749110e-01;   self-cons'cy = 1.8243e-06  [1.8243e-06 for sym-block kappa = -2]

Iteration 16 for symmetries ... 
  1s_1/2::  en [a.u.] = -9.8876005e+00;   self-cons'cy = 3.0982e-08  [1.3049e-


From this multiplet, we just take the *lowest* level (the ground level) in order to calculate the density for the (electronic) potential. In JAC, we have different choices for generating such an **electronic** potential that just arise from the electronic density, and as seen from ? compute at the REPL:

   •    ("radial potential: core-Hartree", grid::Radial.Grid, level::Level) 
       ... to compute a (radial) core-Hartree potential for the given level; 
           cf. Basics.computePotentialCoreHartree. A potential::RadialPotential 
           is returned. 

   •    ("radial potential: Kohn-Sham", grid::Radial.Grid, level::Level) 
       ... to compute a (radial) Kohn-Sham potential for the given level; 
           cf. Basics.computePotentialKohnSham. A potential::RadialPotential is 
           returned.

   •    ("radial potential: Dirac-Fock-Slater", grid::Radial.Grid, level::Level) 
       ... to compute a (radial) Dirac-Fock-Slater potential for the given level; this 
           potential is rather simple but includes some undesired self-interaction and 
           exhibits a wrong asymptotic behaviour. Cf. Basics.computePotentialDFS. 
           A potential::RadialPotential is returned.


In [6]:
level  = multiplet.levels[1]
potCH  = compute("radial potential: core-Hartree", grid, level)
potDFS = compute("radial potential: Dirac-Fock-Slater", grid, level)

coreSubshells = Subshell[1s_1/2, 2s_1/2];    subshells = Subshell[1s_1/2, 2s_1/2, 2p_1/2, 2p_3/2]


DFS (radial) potential ... defined on 392 grid points ...
Zr:    [0.0, 2.6955607059564223e-6, -5.434596604675834e-7, -5.82282910371863e-7, -1.6059227517515662e-6, -4.559757581708367e-6, -6.406366344836484e-6, -6.917100009381071e-6, -6.793141197899899e-6, -7.566326471167558e-6  …  -1.6521180096761336e-5, -1.6997984793507585e-5, -1.840888391790231e-5, -2.1396074388405442e-5, -2.5194990715107997e-5, -2.831799102219741e-5, -3.063234020923076e-5, -3.21115206158104e-5, -3.3027169688644986e-5, -3.533447353281762e-5]  ...  [-6.0, -6.000000000000003, -5.999999999999999, -6.000000000000004, -5.999999999999999, -6.000000000000002, -5.999999999999999, -6.0, -6.000000000000001, -5.999999999999996  …  -6.000000000000004, -6.000000000000001, -6.000000000000003, -6.000000000000005, -5.999999999999999, -5.999999999999999, -6.000000000000002, -6.0000000000000036, -6.0, -5.999999999999998]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 392,  ntL = 69,  ntS = 71, orderL = 7,  orderS = 8,  n


All these potentials are represented as *effective charge* $Z(r)$ and can be added to the nuclear potential, for example:

In [7]:
add(nucPotential, potDFS)

nuclear-potential: Fermi-distributed+DFS (radial) potential ... defined on 392 grid points ...
Zr:    [0.0, 0.015622513768086672, 0.03590526099223162, 0.06043026291944624, 0.08495293134027639, 0.10523281020065203, 0.11777238754107956, 0.12581795181779093, 0.14609385219119195, 0.1788775273906122  …  0.3111128557686238, 0.32311941468563676, 0.35183978014834993, 0.39826148078365287, 0.45434229564729456, 0.5103706313406959, 0.5566687156429055, 0.5852757770687644, 0.6022338661276299, 0.6427823050363154]  ...  [0.0, -8.881784197001252e-16, 1.7763568394002505e-15, -2.6645352591003757e-15, 1.7763568394002505e-15, -1.7763568394002505e-15, 1.7763568394002505e-15, 0.0, 8.881784197001252e-16, 6.217248937900877e-15  …  -2.6645352591003757e-15, 0.0, -8.881784197001252e-16, -3.552713678800501e-15, 1.7763568394002505e-15, 1.7763568394002505e-15, -1.7763568394002505e-15, -2.6645352591003757e-15, 8.881784197001252e-16, 2.6645352591003757e-15]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints =


As seen from this output, the effective charge of the overall atomic potential goes to *zero* as required for a neutral atom. We can display these potentials also graphically by using `JAC.plot()`

# **The following part of this nootebook is still under construction.**